## AIHub Json Parsing

### Development Environment

In [ ]:
%pip install kss==3.7.3

KSS Argument Error: Restart Jupyter Kernel Runtime

In [ ]:
%pip install python-mecab-ko

KSS 3.7.3 matches python-mecab-ko

In [ ]:
%pip install pandas

In [ ]:
%pip install ray

In [ ]:
import re
import os
import kss
import ray
import json
from mecab import MeCab
import pandas as pd
from glob import glob
from itertools import chain

In [2]:
pwd

'c:\\Users\\MinSeok\\Documents\\text-preprocessing\\sentence-segmentation'

### Basic Function

In [6]:
def sorted_list(path_list):
    path_list = sorted(path_list, reverse=False)
    path_list = sorted(path_list, key=len)
    
    return path_list

In [ ]:
def divide_source_file_list(l, n): 
    
  for i in range(0, len(l), n): 
    yield l[i:i + n] 

In [ ]:
def json_file_path_list(path_list):
    
    file_path  = [glob(i, recursive = True) for i in path_list][0]
    file_path = sorted_list(file_path)
    
    return file_path

In [ ]:
def train_valid_json_file_path_list(path_list):

  train_file_path, valid_file_path = [glob(i, recursive = True) if 'rain' in i
                                      else glob(i, recursive = True)
                                      for i in path_list]

  train_file_path = sorted_list(train_file_path)
  valid_file_path = sorted_list(valid_file_path)
    
  return train_file_path, valid_file_path

In [ ]:
def txt_file_path_list(source_file_nested_list, folder_corpus_type_path):

  text_file_path_list = [folder_corpus_type_path + str(i) + ".txt"
                              for i in range(len(source_file_nested_list))]
    
  return text_file_path_list

In [ ]:
def make_topic_json_txt_file_path_list(json_folder_list, topic_name_list, txt_path_list):
    
    json_path_list = []
    the_number_of_file = 0
    the_numer_of_txt_file = 0
    
    for json_folder in json_folder_list:
        for topic_name in topic_name_list:
            json_path_list.append(json_folder + topic_name + '/**/*.json')

    for i in range(len(json_path_list)):
        
        json_path = json_path_list[i]
        
        if 'train'[1:] in json_path:
            train_file_path = glob(json_path, recursive = True) 
            the_number_of_file += len(train_file_path)
            
            if i < 9:
                train_file_list_variable = "train_file_list_" + "0" + str(i + 1)
                txt_file_path_list_variable = "train_txt_file_path_list_" + "0" + str(i + 1)
                
            elif i >= 9:
                train_file_list_variable = "train_file_list_" + str(i + 1)
                txt_file_path_list_variable = "train_txt_file_path_list_" + str(i + 1)
                
            globals()[train_file_list_variable] = train_file_path
            globals()[txt_file_path_list_variable] = txt_file_path_list(train_file_path, 
                                                                        txt_path_list[0] + topic_name_list[i] + "_train_")   
            
        elif 'valid'[1:] in json_path:
            valid_file_path = glob(json_path, recursive = True) 
            the_number_of_file += len(valid_file_path)
            
            if i < 9:
                valid_file_list_variable = "valid_file_list_" + "0" + str(i + 1)
                txt_file_path_list_variable = "valid_txt_file_path_list_" + "0" + str(i + 1)
                
            elif i >= 9:
                valid_file_list_variable = "valid_file_list_" + str(i + 1)
                txt_file_path_list_variable = "valid_txt_file_path_list_" + str(i + 1)
                
            globals()[valid_file_list_variable] = valid_file_nested_list
            globals()[txt_file_path_list_variable] = txt_file_path_list(valid_file_path, 
                                                                        txt_path_list[1] + topic_name_list[i - (len(json_path_list)) // 2] + "_valid_")  

    print("The number of file: ", the_number_of_file)  

In [ ]:
def formal_preprocessing_text(source):
    preprocessing_sentence_list = []
    
    source = source.strip()
    # strip으로 앞뒤 공백 제거

    source = re.sub(r"\[.*?\]|\{.*?\}", "", source)
    # 기타 괄호 제거할 시 괄호 내부에 모든 텍스트 제거


    try:
        bracket_form = re.compile('\(([^)]+)')
        text_in_small_bracket = bracket_form.findall(source)
    
    
        if type(text_in_small_bracket) == str:

            text = text_in_small_bracket

            text_size = len(text)
            last_index = source.find(text) + len(text)
            if len(source) >= last_index+1 and source[last_index-text_size-1] == '(' and source[last_index+1] == '.':
                source = source.replace(source[last_index-text_size-1 : last_index+1] + ".", ".")

            if len(text.split()) > 5 and bool(re.match(r'[.]|[!]|[?]', text[-1])) == True:
                small_bracket = "(" + text + ")"
                source = source.replace(small_bracket, text)    

        elif type(text_in_small_bracket) == list:

            for text in text_in_small_bracket:

                text_size = len(text)
                last_index = source.find(text) + len(text)
                if len(source) >= last_index+1 and source[last_index-text_size-1] == '(' and source[last_index+1] == '.':
                    source = source.replace(source[last_index-text_size-1 : last_index+1] + ".", ".")

                if len(text.split()) > 5 and bool(re.match(r'[.]|[!]|[?]', text[-1])) == True:
                    small_bracket = "(" + text + ")"
                    source = source.replace(small_bracket, text)    

    except:
        pass

        # 마침표(.) 앞에 소괄호')'가 있을시 소괄호 제거와 함께 소괄호 내부 텍스트 제거
        # 소괄호 내부 텍스트가 5어절 이상이고 끝이 온점(.). 느낌표(!). 물음표(?)일 떼 소괄호 제거
        
    
    if bool(re.match(r'[가나다라마바사아자차카타파하]+[.]', source[:2])) == True:
        source = source.replace(source[:2], "")
        
    source = re.sub(r' [가나다라마바사아자차카타파하]+[.]', "", source)
    # '가.', '나.', ... 형태의 문자열 제거  
        
    for sentence in kss.split_sentences(source, use_heuristic=False,
                                        num_workers=32):
    # KSS(Korean Sentence Segmentation)로 문장 분리 
    # Formal articles (wiki, news, essays): recommend to False
    

        if re.search("^[A-Za-z0-9ㄱ-ㅎ가-힣一-鿕㐀-䶵豈-龎]", sentence[0]) is not None and \
            bool(re.match(r'[.]|[!]|[?]', sentence[-1])) == True and \
            len(sentence.split()) > 5:
            # 문장의 시작이 특수문자인 문장(영어 대소문자, 한글, 한자, 숫자, -, + 제외
            # 문장의 끝이 온점(.). 느낌표(!). 물음표(?)가 아닌 문장 제외
            # 다섯 어절 이하 문장 제외


            if ']' in sentence and '[' not in sentence:
                sentence  = re.sub(r".*?]", "", sentence)    
            # 중괄호 앞에 있는 '성명/직함]' 형태 제거


            sentence = re.sub(r"[^A-Za-z0-9ㄱ-ㅎ가-힣一-鿕㐀-䶵豈-龎()+-.,]", " ", sentence)
            # 특수문자 제거(영어 대소문자, 한글, 한자, 숫자, -, +, 소괄호, 마침표, 쉼표, 제외)

            sentence = sentence.strip()
            # strip으로 앞뒤 공백 제거

            total_length = len(sentence.replace(" " , ""))
            hangeul_length = len(re.sub(r"[^ㄱ-ㅣ가-힣\s]", "", sentence.replace(" " , "")))
            hangeul_ratio = hangeul_length / total_length
            if hangeul_ratio >= 0.5:
            # 한글이 아닌 문자열이 50% 이상이 넘은 문장 제외
               
                sentence = sentence.replace(". . .", ".")
                sentence = sentence.replace(". .", ".")
                sentence = sentence.replace("..", ".")
                
            # 구두점 ". . .", ". .", ".."을 온점 "."으로 교체

                for sentence2 in kss.split_sentences(sentence, use_heuristic=False,
                                        num_workers=32):
                    for sentence3 in kss.split_sentences(sentence2, use_heuristic=False,
                                                         num_workers=32):
                        preprocessing_sentence_list.append(sentence3)

            # 마지막에 KSS(Korean Sentence Segmentation)로 문장 분리 2번 실행
            
  
    return preprocessing_sentence_list

### AIHUB 대규모 웹데이터 기반 한국어 말뭉치 데이터

[Source](https://www.aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=624)

#### Convert JSON File to TXT File

In [ ]:
json_folder_list = ['AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Training/원천데이터/TS1/',
             'AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Validation/원천데이터/VS1/']

topic_name_list = ['01_IT_과학', '02_건강', '03_경제', '04_교육', '05_국제', '06_라이프스타일',
                   '07_문화', '08_사건사고', '09_사회일반', '10_산업', '11_스포츠', '12_여성복지',
                   '13_여행레저', '14_연예', '15_정치', '16_지역', '17_취미']

txt_path_list = ["exploration/web_data_based_korean_corpus_data_pro/AIHUB_web_data_based_korean_corpus_data_train_", 
                 "exploration/web_data_based_korean_corpus_data_pro/AIHUB_web_data_based_korean_corpus_data_valid_"]


In [ ]:
make_topic_json_txt_file_path_list(json_folder_list, topic_name_list, txt_path_list)

In [ ]:
def make_source_list(json_sample):
    
    source_list = list(pd.DataFrame(json_sample['SJML']['text'])['content'])
    
    return source_list

In [ ]:
def count_number_of_txt_file_with_batch_list(source_file_list, batch_size):
    
    the_number_of_total_txt_file = 0
    the_number_of_txt_file_list = []
    
    for i in range(len(source_file_list)):    
        
        source_file = source_file_list[i]
            
        with open(source_file, 'r', encoding='utf-8') as one_json_file:
            one_json_sample = json.load(one_json_file) 

        source_list = make_source_list(one_json_sample)

        the_number_of_txt_file = ((len(source_list) // batch_size) + 1)

        if len(source_list) >= batch_size:
            print("File:", source_file)    
            print("Index:", i, "  ", "Length of Source List:", len(source_list), \
                "  ", "The number of txt file:", the_number_of_txt_file, "\n")
            the_number_of_txt_file_list.append(the_number_of_txt_file)
            the_number_of_total_txt_file  += the_number_of_txt_file
        else:
            the_number_of_total_txt_file  += 1
            the_number_of_txt_file_list.append(1)
            print("[For Example]")
            print("This is not subject of batch. It's small source list.")                            
            print("File:", source_file)
            print("Length of Source List:", len(source_list), 
                    "  ", "The number of txt file:", 1, "\n") 


    print("Batch Size:", batch_size)
    print("The number of txt file:", the_number_of_total_txt_file)
    
    return the_number_of_total_txt_file, the_number_of_txt_file_list

In [ ]:
def write_jsontext_to_txt_file_with_batch_list(source_file_list,
                                    text_file_path_list,
                                    batch_size, the_number_of_total_txt_file_list):


  print("[Size]")
  print("The number of preprocessing corpus: " + str(sum(the_number_of_txt_file_list)))
  print("\n[Order]")
  num = 0
  
  for i in range(len(source_file_list)):

    source_file = source_file_list[i]

    with open(source_file, 'r', encoding='utf-8') as one_json_file:
      one_json_sample = json.load(one_json_file)

    source_list = make_source_list(one_json_sample)
        
    n = batch_size
    source_batch_list = list(divide_source_file_list(source_list, n))
      
    for source_list in source_batch_list:
        num += 1
        print(str(num), end=" ")  
        
        with open(os.path.join('AIHUB_corpus/' + text_file_path_list[i][:-4] + "_" + str(num) + ".txt"), "a", encoding='utf-8') as fp:        
            fp.write("\n".join(source_list))   
    

In [ ]:
batch_size = 1000
the_number_of_train_txt_file_01, the_number_of_train_txt_file_list_01 = count_number_of_txt_file_with_batch_list(train_file_list_01, batch_size)

In [ ]:
batch_size = 1000
the_number_of_train_txt_file_02, the_number_of_train_txt_file_list_02 = count_number_of_txt_file_with_batch_list(train_file_list_02, batch_size)

In [ ]:
batch_size = 1000
the_number_of_train_txt_file_03, the_number_of_train_txt_file_list_03 = count_number_of_txt_file_with_batch_list(train_file_list_03, batch_size)

In [ ]:
batch_size = 1000
the_number_of_train_txt_file_04, the_number_of_train_txt_file_list_04 = count_number_of_txt_file_with_batch_list(train_file_list_04, batch_size)

In [ ]:
batch_size = 1000
the_number_of_train_txt_file_05, the_number_of_train_txt_file_list_05 = count_number_of_txt_file_with_batch_list(train_file_list_05, batch_size)

In [ ]:
batch_size = 1000
the_number_of_train_txt_file_06, the_number_of_train_txt_file_list_06 = count_number_of_txt_file_with_batch_list(train_file_list_06, batch_size)

In [ ]:
batch_size = 1000
the_number_of_train_txt_file_07, the_number_of_train_txt_file_list_07 = count_number_of_txt_file_with_batch_list(train_file_list_07, batch_size)

In [ ]:
batch_size = 1000
the_number_of_train_txt_file_08, the_number_of_train_txt_file_list_08 = count_number_of_txt_file_with_batch_list(train_file_list_08, batch_size)

In [ ]:
batch_size = 1000
the_number_of_train_txt_file_09, the_number_of_train_txt_file_list_09 = count_number_of_txt_file_with_batch_list(train_file_list_09, batch_size)

In [ ]:
batch_size = 1000
the_number_of_train_txt_file_10, the_number_of_train_txt_file_list_10 = count_number_of_txt_file_with_batch_list(train_file_list_10, batch_size)

In [ ]:
batch_size = 1000
the_number_of_train_txt_file_11, the_number_of_train_txt_file_list_11 = count_number_of_txt_file_with_batch_list(train_file_list_11, batch_size)

In [ ]:
batch_size = 1000
the_number_of_train_txt_file_12, the_number_of_train_txt_file_list_12 = count_number_of_txt_file_with_batch_list(train_file_list_12, batch_size)

In [ ]:
batch_size = 1000
the_number_of_train_txt_file_13, the_number_of_train_txt_file_list_13 = count_number_of_txt_file_with_batch_list(train_file_list_13, batch_size)

In [ ]:
batch_size = 1000
the_number_of_train_txt_file_14, the_number_of_train_txt_file_list_14 = count_number_of_txt_file_with_batch_list(train_file_list_14, batch_size)

In [ ]:
batch_size = 1000
the_number_of_train_txt_file_15, the_number_of_train_txt_file_list_15 = count_number_of_txt_file_with_batch_list(train_file_list_15, batch_size)

In [ ]:
batch_size = 1000
the_number_of_train_txt_file_16, the_number_of_train_txt_file_list_16 = count_number_of_txt_file_with_batch_list(train_file_list_16, batch_size)

In [ ]:
batch_size = 1000
the_number_of_train_txt_file_17, the_number_of_train_txt_file_list_17 = count_number_of_txt_file_with_batch_list(train_file_list_17, batch_size)

In [ ]:
batch_size = 1000
the_number_of_valid_txt_file_01, the_number_of_valid_txt_file_list_01 = count_number_of_txt_file_with_batch_listvalid_file_list_01, batch_size)

In [ ]:
batch_size = 1000
the_number_of_valid_txt_file_02, the_number_of_valid_txt_file_list_02 = count_number_of_txt_file_with_batch_list(valid_file_list_02, batch_size)

In [ ]:
batch_size = 1000
the_number_of_valid_txt_file_03, the_number_of_valid_txt_file_list_03 = count_number_of_txt_file_with_batch_list(valid_file_list_03, batch_size)

In [ ]:
batch_size = 1000
the_number_of_valid_txt_file_04, the_number_of_valid_txt_file_list_04 = count_number_of_txt_file_with_batch_list(valid_file_list_04, batch_size)

In [ ]:
batch_size = 1000
the_number_of_valid_txt_file_05, the_number_of_valid_txt_file_list_05 = count_number_of_txt_file_with_batch_list(valid_file_list_05, batch_size)

In [ ]:
batch_size = 1000
the_number_of_valid_txt_file_06, the_number_of_valid_txt_file_list_06 = count_number_of_txt_file_with_batch_list(valid_file_list_06, batch_size)

In [ ]:
batch_size = 1000
the_number_of_valid_txt_file_07, the_number_of_valid_txt_file_list_07 = count_number_of_txt_file_with_batch_list(valid_file_list_07, batch_size)

In [ ]:
batch_size = 1000
the_number_of_valid_txt_file_08, the_number_of_valid_txt_file_list_08 = count_number_of_txt_file_with_batch_list(valid_file_list_08, batch_size)

In [ ]:
batch_size = 1000
the_number_of_valid_txt_file_09, the_number_of_valid_txt_file_list_09 = count_number_of_txt_file_with_batch_list(valid_file_list_09, batch_size)

In [ ]:
batch_size = 1000
the_number_of_valid_txt_file_10, the_number_of_valid_txt_file_list_10 = count_number_of_txt_file_with_batch_list(valid_file_list_10, batch_size)

In [ ]:
batch_size = 1000
the_number_of_valid_txt_file_11, the_number_of_valid_txt_file_list_11 = count_number_of_txt_file_with_batch_list(valid_file_list_11, batch_size)

In [ ]:
batch_size = 1000
the_number_of_valid_txt_file_12, the_number_of_valid_txt_file_list_12 = count_number_of_txt_file_with_batch_listvalid_file_list_12, batch_size)

In [ ]:
batch_size = 1000
the_number_of_valid_txt_file_13, the_number_of_valid_txt_file_list_13 = count_number_of_txt_file_with_batch_list(valid_file_list_13, batch_size)

In [ ]:
batch_size = 1000
the_number_of_valid_txt_file_14, the_number_of_valid_txt_file_list_14 = count_number_of_txt_file_with_batch_list(valid_file_list_14, batch_size)

In [ ]:
batch_size = 1000
the_number_of_valid_txt_file_15, the_number_of_valid_txt_file_list_15 = count_number_of_txt_file_with_batch_list(valid_file_list_15, batch_size)

In [ ]:
batch_size = 1000
the_number_of_valid_txt_file_16, the_number_of_valid_txt_file_list_16 = count_number_of_txt_file_with_batch_list(valid_file_list_16, batch_size)

In [ ]:
batch_size = 1000
the_number_of_valid_txt_file_17, the_number_of_valid_txt_file_list_17 = count_number_of_txt_file_with_batch_list(valid_file_list_17, batch_size)

In [ ]:
batch_size = 1000
write_jsontext_to_txt_file_with_batch_list(train_file_list_01, train_text_file_path_list_01, batch_size, the_number_of_train_txt_file_list_01)

In [ ]:
batch_size = 1000
write_jsontext_to_txt_file_with_batch_list(train_file_list_02, train_text_file_path_list_02, batch_size, the_number_of_train_txt_file_list_02)

In [ ]:
batch_size = 1000
write_jsontext_to_txt_file_with_batch_list(train_file_list_03, train_text_file_path_list_03, batch_size, the_number_of_train_txt_file_list_03)

In [ ]:
batch_size = 1000
write_jsontext_to_txt_file_with_batch_list(train_file_list_04, train_text_file_path_list_04, batch_size, the_number_of_train_txt_file_list_04)

In [ ]:
batch_size = 1000
write_jsontext_to_txt_file_with_batch_list(train_file_list_05, train_text_file_path_list_05, batch_size, the_number_of_train_txt_file_list_05)

In [ ]:
batch_size = 1000
write_jsontext_to_txt_file_with_batch_list(train_file_list_06, train_text_file_path_list_06, batch_size, the_number_of_train_txt_file_list_06)

In [ ]:
batch_size = 1000
write_jsontext_to_txt_file_with_batch_list(train_file_list_07, train_text_file_path_list_07, batch_size, the_number_of_train_txt_file_list_07)

In [ ]:
batch_size = 1000
write_jsontext_to_txt_file_with_batch_list(train_file_list_08, train_text_file_path_list_08, batch_size, the_number_of_train_txt_file_list_08)

In [ ]:
batch_size = 1000
write_jsontext_to_txt_file_with_batch_list(train_file_list_09, train_text_file_path_list_09, batch_size, the_number_of_train_txt_file_list_09)

In [ ]:
batch_size = 1000
write_jsontext_to_txt_file_with_batch_list(train_file_list_10, train_text_file_path_list_10, batch_size, the_number_of_train_txt_file_list_10)

In [ ]:
batch_size = 1000
write_jsontext_to_txt_file_with_batch_list(train_file_list_11, train_text_file_path_list_11, batch_size, the_number_of_train_txt_file_list_11)

In [ ]:
batch_size = 1000
write_jsontext_to_txt_file_with_batch_list(train_file_list_12, train_text_file_path_list_12, batch_size, the_number_of_train_txt_file_list_12)

In [ ]:
batch_size = 1000
write_jsontext_to_txt_file_with_batch_list(train_file_list_13, train_text_file_path_list_13, batch_size, the_number_of_train_txt_file_list_13)

In [ ]:
batch_size = 1000
write_jsontext_to_txt_file_with_batch_list(train_file_list_14, train_text_file_path_list_14, batch_size, the_number_of_train_txt_file_list_14)

In [ ]:
batch_size = 1000
write_jsontext_to_txt_file_with_batch_list(train_file_list_15, train_text_file_path_list_15, batch_size, the_number_of_train_txt_file_list_15)

In [ ]:
batch_size = 1000
write_jsontext_to_txt_file_with_batch_list(train_file_list_16, train_text_file_path_list_16, batch_size, the_number_of_train_txt_file_list_16)

In [ ]:
batch_size = 1000
write_jsontext_to_txt_file_with_batch_list(train_file_list_17, train_text_file_path_list_17, batch_size, the_number_of_train_txt_file_list_17)

In [ ]:
batch_size = 1000
write_jsontext_to_txt_file_with_batch_list(valid_file_list_01, valid_text_file_path_list_01, batch_size, the_number_of_valid_txt_file_list_01)

In [ ]:
batch_size = 1000
write_jsontext_to_txt_file_with_batch_list(valid_file_list_02, valid_text_file_path_list_02, batch_size, the_number_of_valid_txt_file_list_02)

In [ ]:
batch_size = 1000
write_jsontext_to_txt_file_with_batch_list(valid_file_list_03, valid_text_file_path_list_03, batch_size, the_number_of_valid_txt_file_list_03)

In [ ]:
batch_size = 1000
write_jsontext_to_txt_file_with_batch_list(valid_file_list_04, valid_text_file_path_list_04, batch_size, the_number_of_valid_txt_file_list_04)

In [ ]:
batch_size = 1000
write_jsontext_to_txt_file_with_batch_list(valid_file_list_05, valid_text_file_path_list_05, batch_size, the_number_of_valid_txt_file_list_05)

In [ ]:
batch_size = 1000
write_jsontext_to_txt_file_with_batch_list(valid_file_list_06, valid_text_file_path_list_06, batch_size, the_number_of_valid_txt_file_list_06)

In [ ]:
batch_size = 1000
write_jsontext_to_txt_file_with_batch_list(valid_file_list_07, valid_text_file_path_list_07, batch_size, the_number_of_valid_txt_file_list_07)

In [ ]:
batch_size = 1000
write_jsontext_to_txt_file_with_batch_list(valid_file_list_08, valid_text_file_path_list_08, batch_size, the_number_of_valid_txt_file_list_08)

In [ ]:
batch_size = 1000
write_jsontext_to_txt_file_with_batch_list(valid_file_list_09, valid_text_file_path_list_09, batch_size, the_number_of_valid_txt_file_list_09)

In [ ]:
batch_size = 1000
write_jsontext_to_txt_file_with_batch_list(valid_file_list_10, valid_text_file_path_list_10, batch_size, the_number_of_valid_txt_file_list_10)

In [ ]:
batch_size = 1000
write_jsontext_to_txt_file_with_batch_list(valid_file_list_11, valid_text_file_path_list_11, batch_size, the_number_of_valid_txt_file_list_11)

In [ ]:
batch_size = 1000
write_jsontext_to_txt_file_with_batch_list(valid_file_list_12, valid_text_file_path_list_12, batch_size, the_number_of_valid_txt_file_list_12)

In [ ]:
batch_size = 1000
write_jsontext_to_txt_file_with_batch_list(valid_file_list_13, valid_text_file_path_list_13, batch_size, the_number_of_valid_txt_file_list_13)

In [ ]:
batch_size = 1000
write_jsontext_to_txt_file_with_batch_list(valid_file_list_14, valid_text_file_path_list_14, batch_size, the_number_of_valid_txt_file_list_14)

In [ ]:
batch_size = 1000
write_jsontext_to_txt_file_with_batch_list(valid_file_list_15, valid_text_file_path_list_15, batch_size, the_number_of_valid_txt_file_list_15)

In [ ]:
batch_size = 1000
write_jsontext_to_txt_file_with_batch_list(valid_file_list_16, valid_text_file_path_list_16, batch_size, the_number_of_valid_txt_file_list_16)

In [ ]:
batch_size = 1000
write_jsontext_to_txt_file_with_batch_list(valid_file_list_17, valid_text_file_path_list_17, batch_size, the_number_of_valid_txt_file_list_17)

#### Preprocess TXT File

In [3]:
def post_txt_file_path_list(corpus_path_list):

  post_corpus_path_list = [corpus_file.replace("pro", "post")
                      for corpus_file in corpus_path_list]

  return post_corpus_path_list

In [4]:
def make_pro_post_txt_file_path_list(pro_corpus_path):
    
    pro_total_corpus_path_list = glob(pro_corpus_path)
    pro_total_corpus_path_list = sorted_list(pro_total_corpus_path_list)
    post_total_corpus_path_list = post_txt_file_path_list(pro_total_corpus_path_list)

    return pro_total_corpus_path_list, post_total_corpus_path_list

In [8]:
pro_corpus_path = "AIHUB_corpus/exploration/web_data_based_korean_corpus_data_pro/AIHUB_web_data_based_korean_corpus_data_" + "*.txt"
pro_total_corpus_path_list, post_total_corpus_path_list = make_pro_post_txt_file_name_list(pro_corpus_path)

In [ ]:
len(pro_total_corpus_path_list)

In [ ]:
line_list = []
line_num = 0
with open(pro_total_corpus_path_list[0], 'r', encoding='utf-8') as f:
    lines = f.read().splitlines() 
    for line in lines:
        line_num += 1
        if line_num <= 1:
           line_list.append(line)
for line in line_list:
    print(line, end="\n\n")

In [ ]:
line_list = []
line_num = 0
with open(pro_total_corpus_path_list[0], 'r', encoding='utf-8') as f:
    lines = f.read().splitlines()
    for line in lines:
        line_num += 1
        if line_num <= 1:  
            sentences = formal_preprocessing_text(line)
            for sentence in sentences:
                line_list.append(sentence) 
            
for line in line_list:
    print(line, end="\n\n")

In [179]:
ray.init(num_cpus = 4)

@ray.remote
def formal_preprocessing_text(source):
    preprocessing_sentence_list = []
    
    source = source.strip()
    # strip으로 앞뒤 공백 제거

    source = re.sub(r"\[.*?\]|\{.*?\}", "", source)
    # 기타 괄호 제거할 시 괄호 내부에 모든 텍스트 제거


    try:
        bracket_form = re.compile('\(([^)]+)')
        text_in_small_bracket = bracket_form.findall(source)
    
    
        if type(text_in_small_bracket) == str:

            text = text_in_small_bracket

            text_size = len(text)
            last_index = source.find(text) + len(text)
            if len(source) >= last_index+1 and source[last_index-text_size-1] == '(' and source[last_index+1] == '.':
                source = source.replace(source[last_index-text_size-1 : last_index+1] + ".", ".")

            if len(text.split()) > 5 and bool(re.match(r'[.]|[!]|[?]', text[-1])) == True:
                small_bracket = "(" + text + ")"
                source = source.replace(small_bracket, text)    

        elif type(text_in_small_bracket) == list:

            for text in text_in_small_bracket:

                text_size = len(text)
                last_index = source.find(text) + len(text)
                if len(source) >= last_index+1 and source[last_index-text_size-1] == '(' and source[last_index+1] == '.':
                    source = source.replace(source[last_index-text_size-1 : last_index+1] + ".", ".")

                if len(text.split()) > 5 and bool(re.match(r'[.]|[!]|[?]', text[-1])) == True:
                    small_bracket = "(" + text + ")"
                    source = source.replace(small_bracket, text)    

    except:
        pass

        # 마침표(.) 앞에 소괄호')'가 있을시 소괄호 제거와 함께 소괄호 내부 텍스트 제거
        # 소괄호 내부 텍스트가 5어절 이상이고 끝이 온점(.). 느낌표(!). 물음표(?)일 떼 소괄호 제거
        
    
    if bool(re.match(r'[가나다라마바사아자차카타파하]+[.]', source[:2])) == True:
        source = source.replace(source[:2], "")
        
    source = re.sub(r' [가나다라마바사아자차카타파하]+[.]', "", source)
    # '가.', '나.', ... 형태의 문자열 제거 
        
    for sentence in kss.split_sentences(source, use_heuristic=False,
                                        num_workers=32):
    # KSS(Korean Sentence Segmentation)로 문장 분리 
    # Formal articles (wiki, news, essays): recommend to False
    

        if re.search("^[A-Za-z0-9ㄱ-ㅎ가-힣一-鿕㐀-䶵豈-龎]", sentence[0]) is not None and \
            bool(re.match(r'[.]|[!]|[?]', sentence[-1])) == True and \
            len(sentence.split()) > 5:
            # 문장의 시작이 특수문자인 문장(영어 대소문자, 한글, 한자, 숫자, -, + 제외
            # 문장의 끝이 온점(.). 느낌표(!). 물음표(?)가 아닌 문장 제외
            # 다섯 어절 이하 문장 제외


            if ']' in sentence and '[' not in sentence:
                sentence  = re.sub(r".*?]", "", sentence)    
            # 중괄호 앞에 있는 '성명/직함]' 형태 제거


            sentence = re.sub(r"[^A-Za-z0-9ㄱ-ㅎ가-힣一-鿕㐀-䶵豈-龎()+-.,]", " ", sentence)
            # 특수문자 제거(영어 대소문자, 한글, 한자, 숫자, -, +, 소괄호, 마침표, 쉼표, 제외)

            sentence = sentence.strip()
            # strip으로 앞뒤 공백 제거

            total_length = len(sentence.replace(" " , ""))
            hangeul_length = len(re.sub(r"[^ㄱ-ㅣ가-힣\s]", "", sentence.replace(" " , "")))
            hangeul_ratio = hangeul_length / total_length
            if hangeul_ratio >= 0.5:
            # 한글이 아닌 문자열이 50% 이상이 넘은 문장 제외
            
                sentence = sentence.replace(". . .", ".")
                sentence = sentence.replace(". .", ".")
                sentence = sentence.replace("..", ".")
                
            # 구두점 ". . .", ". .", ".."을 온점 "."으로 교체

                for sentence2 in kss.split_sentences(sentence, use_heuristic=False,
                                        num_workers=32):
                    for sentence3 in kss.split_sentences(sentence2, use_heuristic=False,
                                                         num_workers=32):
                        preprocessing_sentence_list.append(sentence3)

            # 마지막에 KSS(Korean Sentence Segmentation)로 문장 분리 2번 실행

  
    return preprocessing_sentence_list

2023-05-29 18:06:33,632	INFO worker.py:1625 -- Started a local Ray instance.


In [ ]:
def preprocessing_corpus_txt(pro_total_corpus_path_list, post_total_corpus_path_list):
    print("[Size]")
    print("The number of preprocessing corpus: " + str(len(pro_total_corpus_path_list)))
    print("\n[Order]")
    num = 0
    process_num = 10    

    for pro, post in zip(pro_total_corpus_path_list, post_total_corpus_path_list):

        sentence_list = []

        with open(pro, 'r', encoding='utf-8') as f:
            lines = f.read().splitlines() 
            nested_lines_num = len(lines) // process_num
            for i in range(nested_lines_num - 1):
                start_line = process_num * i
                end_line = process_num * (i+1)
                futures = [formal_preprocessing_text.remote(lines[start_line:end_line][j]) for j in range(process_num)]
                results = ray.get(futures)

                if i == nested_lines_num - 2:
                    futures = [formal_preprocessing_text.remote(lines[end_line:][j]) for j in range(len(lines) - end_line)]
                    results = ray.get(futures)

                sentences = list(chain.from_iterable(results))
                sentence_list.append(sentences)

        sentence_list = list(chain.from_iterable(sentence_list))

        num += 1
        print(str(num), end=" ")  

        with open(post, 'a', encoding='utf-8') as fp:
            fp.write("\n".join(sentence_list))

In [ ]:
def make_topic_pro_post_txt_file_name_list(pro_corpus_path, topic_name_list):
    

    for i in range(len(topic_name_list)):
 
        pro_total_corpus_path_list = sorted_list(glob(pro_corpus_path + topic_name_list[i] + "_train_" + "*.txt"))
        post_total_corpus_path_list = post_txt_file_name_list(pro_total_corpus_path_list)
        
        if i < 9:
            pro_total_corpus_path_list_variable = "pro_total_corpus_path_list_" + "train_" + "0" + str(i + 1)
            post_total_corpus_path_list_variable = "post_total_corpus_path_list_" + "train_" + "0" + str(i + 1)
        
        elif i >= 9:
            pro_total_corpus_path_list_variable = "pro_total_corpus_path_list_" + "train_" + str(i + 1)
            post_total_corpus_path_list_variable = "post_total_corpus_path_list_" + "train_" + str(i + 1)
            
        globals()[pro_total_corpus_path_list_variable] = pro_total_corpus_path_list
        globals()[post_total_corpus_path_list_variable] = post_total_corpus_path_list
        
        
        pro_total_corpus_path_list = sorted_list(glob(pro_corpus_path + topic_name_list[i] + "_valid_" + "*.txt"))
        post_total_corpus_path_list = post_txt_file_name_list(pro_total_corpus_path_list)
        
        if i < 9:
            pro_total_corpus_path_list_variable = "pro_total_corpus_path_list_" + "valid_" + "0" + str(i + 1)
            post_total_corpus_path_list_variable = "post_total_corpus_path_list_" + "valid_" + "0" + str(i + 1)
        
        elif i >= 9:
            pro_total_corpus_path_list_variable = "pro_total_corpus_path_list_" + "valid_" + str(i + 1)
            post_total_corpus_path_list_variable = "post_total_corpus_path_list_" + "valid_" + str(i + 1)
            
        globals()[pro_total_corpus_path_list_variable] = pro_total_corpus_path_list
        globals()[post_total_corpus_path_list_variable] = post_total_corpus_path_list

In [180]:
pro_corpus_name = "AIHUB_corpus/exploration/web_data_based_korean_corpus_data_pro/AIHUB_web_data_based_korean_corpus_data_"

topic_name_list = ['01_IT_과학', '02_건강', '03_경제', '04_교육', '05_국제', '06_라이프스타일', '07_문화',
                  '08_사건사고', '09_사회일반', '10_산업', '11_스포츠', '12_여성복지', '13_여행레저',
                  '14_연예', '15_정치', '16_지역', '17_취미']

In [ ]:
make_topic_pro_post_txt_file_name_list(pro_corpus_path, topic_name_list)

In [ ]:
preprocessing_corpus_txt(pro_total_corpus_path_list_train_01, post_total_corpus_path_list_train_01)

[Size]
The number of preprocessing corpus: 1217

[Order]
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 

In [ ]:
preprocessing_corpus_txt(pro_total_corpus_path_list_train_02, post_total_corpus_path_list_train_02)

In [ ]:
preprocessing_corpus_txt(pro_total_corpus_path_list_train_03, post_total_corpus_path_list_train_03)

In [ ]:
preprocessing_corpus_txt(pro_total_corpus_path_list_train_04, post_total_corpus_path_list_train_04)

In [ ]:
preprocessing_corpus_txt(pro_total_corpus_path_list_train_05, post_total_corpus_path_list_train_05)

In [ ]:
preprocessing_corpus_txt(pro_total_corpus_path_list_train_06, post_total_corpus_path_list_train_06)

In [ ]:
preprocessing_corpus_txt(pro_total_corpus_path_list_train_07, post_total_corpus_path_list_train_07)

In [ ]:
preprocessing_corpus_txt(pro_total_corpus_path_list_train_08, post_total_corpus_path_list_train_08)

In [ ]:
preprocessing_corpus_txt(pro_total_corpus_path_list_train_09, post_total_corpus_path_list_train_09)

In [ ]:
preprocessing_corpus_txt(pro_total_corpus_path_list_train_10, post_total_corpus_path_list_train_10)

In [ ]:
preprocessing_corpus_txt(pro_total_corpus_path_list_train_11, post_total_corpus_path_list_train_11)

In [ ]:
preprocessing_corpus_txt(pro_total_corpus_path_list_train_12, post_total_corpus_path_list_train_12)

In [ ]:
preprocessing_corpus_txt(pro_total_corpus_path_list_train_13, post_total_corpus_path_list_train_13)

In [ ]:
preprocessing_corpus_txt(pro_total_corpus_path_list_train_14, post_total_corpus_path_list_train_14)

In [ ]:
preprocessing_corpus_txt(pro_total_corpus_path_list_train_15, post_total_corpus_path_list_train_15)

In [ ]:
preprocessing_corpus_txt(pro_total_corpus_path_list_train_16, post_total_corpus_path_list_train_16)

In [ ]:
preprocessing_corpus_txt(pro_total_corpus_path_list_train_17, post_total_corpus_path_list_train_17)

In [ ]:
preprocessing_corpus_txt.(pro_total_corpus_path_list_valid_01, post_total_corpus_path_list_valid_01)

In [ ]:
preprocessing_corpus_txt(pro_total_corpus_path_list_valid_02, post_total_corpus_path_list_valid_02)

In [ ]:
preprocessing_corpus_txt(pro_total_corpus_path_list_valid_03, post_total_corpus_path_list_valid_03)

In [ ]:
preprocessing_corpus_txt(pro_total_corpus_path_list_valid_04, post_total_corpus_path_list_valid_04)

In [ ]:
preprocessing_corpus_txt(pro_total_corpus_path_list_valid_05, post_total_corpus_path_list_valid_05)

In [ ]:
preprocessing_corpus_txt(pro_total_corpus_path_list_valid_06, post_total_corpus_path_list_valid_06)

In [ ]:
preprocessing_corpus_txt(pro_total_corpus_path_list_valid_07, post_total_corpus_path_list_valid_07)

In [ ]:
preprocessing_corpus_txt(pro_total_corpus_path_list_valid_08, post_total_corpus_path_list_valid_08)

In [ ]:
preprocessing_corpus_txt(pro_total_corpus_path_list_valid_09, post_total_corpus_path_list_valid_09)

In [ ]:
preprocessing_corpus_txt(pro_total_corpus_path_list_valid_10, post_total_corpus_path_list_valid_10)

In [ ]:
preprocessing_corpus_txt(pro_total_corpus_path_list_valid_11, post_total_corpus_path_list_valid_11)

In [ ]:
preprocessing_corpus_txt(pro_total_corpus_path_list_valid_12, post_total_corpus_path_list_valid_12)

In [ ]:
preprocessing_corpus_txt(pro_total_corpus_path_list_valid_13, post_total_corpus_path_list_valid_13)

In [ ]:
preprocessing_corpus_txt(pro_total_corpus_path_list_valid_14, post_total_corpus_path_list_valid_14)

In [ ]:
preprocessing_corpus_txt(pro_total_corpus_path_list_valid_15, post_total_corpus_path_list_valid_15)

In [ ]:
preprocessing_corpus_txt(pro_total_corpus_path_list_valid_16, post_total_corpus_path_list_valid_16)

In [ ]:
preprocessing_corpus_txt(pro_total_corpus_path_list_valid_17, post_total_corpus_path_list_valid_17)

In [178]:
ray.shutdown()

In [ ]:
def merge_and_deduplicate_topic_corpus_txt(preprocessing_corpus_path, merge_corpus_path,
                                      deduplicate_corpus_path):
    
    corpus_list = glob(preprocessing_corpus_path)
    corpus_list = sorted_list(corpus_list)
    
    merge_corpus_list = glob(merge_corpus_path + "*.txt")
    
    for i in range(len(topic_name_list)):
        with open(merge_corpus_path + topic_name_list[i] + '.txt', 'w', encoding='utf-8') as f:
            topic_corpus_list_with_none = [j if topic_name_list[i] in j else None for j in corpus_list]
            topic_corpus_list = list(filter(None, topic_corpus_list))
                    for corpus in topic_corpus_list:
                        with open(corpus, encoding='utf-8') as text:
                            for line in text:
                                f.write(line)
                    
    with open(deduplicate_corpus_path, 'w', encoding='utf-8') as f1:
        for corpus in merge_corpus_list:
            with open(corpus, encoding='utf-8') as f2:
                lines = f2.read().splitlines()
                single_sentence_dict = dict.fromkeys(lines)
                single_sentence_list = list(single_sentence_dict)
                f1.write("\n".join(single_sentence_list))                         

In [ ]:
preprocessing_corpus_path = "AIHUB_corpus/exploration/web_data_based_korean_corpus_data_post/AIHUB_web_data_based_korean_corpus_data_" +"*.txt"
merge_corpus_path = 'AIHUB_corpus/duplicate/AIHUB_web_data_based_korean_corpus_data_'
deduplicate_corpus_path = 'AIHUB_corpus/AIHUB_web_data_based_korean_corpus_data.txt'

topic_name_list = ['01_IT_과학', '02_건강', '03_경제', '04_교육', '05_국제', '06_라이프스타일',
                   '07_문화', '08_사건사고', '09_사회일반', '10_산업', '11_스포츠', '12_여성복지',
                   '13_여행레저', '14_연예', '15_정치', '16_지역', '17_취미']

In [ ]:
merge_and_deduplicate_topic_corpus_txt(preprocessing_corpus_path, merge_corpus_path,
                                        deduplicate_corpus_path, topic_name_list)